In [1]:
# Load EDA Pkgs
import pandas as pd
import numpy as np

In [2]:
# Load Data Viz Pkgs
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# ML Pkgs
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report

In [4]:
### Split Dataset into Train and Text
from sklearn.model_selection import train_test_split
# Feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Multi Label Pkgs
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [6]:
train_path = "/kaggle/input/code-injection-6labels-balance/code_injection_6labels_balance.csv"


In [7]:
# Load Dataset
df = pd.read_csv(train_path)

In [8]:
target_list = ["000 - Normal", '126 - Path Traversal',
               '242 - Code Injection', '274 - HTTP Verb Tampering',
               '66 - SQL Injection', '88 - OS Command Injection']

In [9]:
df

,Unnamed: 0,text,000 - Normal,126 - Path Traversal,242 - Code Injection,274 - HTTP Verb Tampering,66 - SQL Injection,88 - OS Command Injection
0,0,GET /,1,0,0,0,0,0
1,1,GET /blog/index.php/2020/04/04/voluptatum-repr...,1,0,0,0,0,0
2,2,GET /blog/xmlrpc.php?rsd,1,0,0,0,0,0
3,3,POST /blog/index.php/my-account/user-logout/?_...,0,0,0,0,0,1
4,4,GET /blog/index.php/2020/04/04/nihil-tenetur-e...,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
69730,69730,POST /blog/index.php/my-account/user-logout/?_...,0,1,0,0,0,1
69731,69731,POST /blog/index.php/my-account/user-logout/?_...,0,1,0,0,0,1
69732,69732,POST /blog/index.php/my-account/user-logout/?_...,0,1,0,0,0,1
69733,69733,POST /blog/index.php/my-account/user-logout/?_...,0,1,0,0,0,1


In [10]:
!pip install neattext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 4.6 MB/s eta 0:00:00


In [11]:
import neattext as nt
import neattext.functions as nfx

In [12]:
# Explore For Noise
df['text'].apply(lambda x:nt.TextFrame(x).noise_scan())

0        {'text_noise': 40.0, 'text_length': 5, 'noise_...
1        {'text_noise': 17.28395061728395, 'text_length...
2        {'text_noise': 20.833333333333336, 'text_lengt...
3        {'text_noise': 8.996539792387544, 'text_length...
4        {'text_noise': 22.58064516129032, 'text_length...
                               ...                        
69730    {'text_noise': 8.58085808580858, 'text_length'...
69731    {'text_noise': 9.24092409240924, 'text_length'...
69732    {'text_noise': 9.30232558139535, 'text_length'...
69733    {'text_noise': 8.49673202614379, 'text_length'...
69734    {'text_noise': 16.071428571428573, 'text_lengt...
Name: text, Length: 69735, dtype: object

In [13]:
# Explore For Noise
df['text'].apply(lambda x:nt.TextExtractor(x).extract_stopwords())

0        [get]
1        [get]
2        [get]
3           []
4        [get]
         ...  
69730       []
69731       []
69732       []
69733       []
69734    [get]
Name: text, Length: 69735, dtype: object

In [14]:
# Explore For Noise
df['text'].apply(nfx.remove_stopwords)

0                                                        /
1        /blog/index.php/2020/04/04/voluptatum-reprehen...
2                                     /blog/xmlrpc.php?rsd
3        POST /blog/index.php/my-account/user-logout/?_...
4        /blog/index.php/2020/04/04/nihil-tenetur-et-ad...
                               ...                        
69730    POST /blog/index.php/my-account/user-logout/?_...
69731    POST /blog/index.php/my-account/user-logout/?_...
69732    POST /blog/index.php/my-account/user-logout/?_...
69733    POST /blog/index.php/my-account/user-logout/?_...
69734    /blog/index.php/my-account/edit-profile%28?mod...
Name: text, Length: 69735, dtype: object

In [15]:
corpus = df['text'].apply(nfx.remove_stopwords)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer = CountVectorizer()

In [17]:
# Build Features
Xfeatures = vectorizer.fit_transform(corpus)

In [18]:
len(vectorizer.get_feature_names_out())

21634

In [19]:
print(Xfeatures.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
len(Xfeatures.toarray()[0])

21634

In [21]:
y = df[target_list]

In [22]:
# Split Data 
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)

In [23]:
### Problem Transform
import skmultilearn

In [24]:
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    # Create an Instance
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    # Predict
    clf_predictions = clf.predict(xtest)
    # Check For Accuracy
    evaluate_model(ytest, clf_predictions)
    acc = accuracy_score(ytest,clf_predictions)
    ham = hamming_loss(ytest,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham}
    return result

In [25]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

In [26]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    y_true1 = y_true.to_numpy()
    y_pred1 = y_pred.toarray()
    acc_list = []
    
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true1[i])[0] )
        set_pred = set( np.where(y_pred1[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [27]:
clf = BinaryRelevance(MultinomialNB())
clf.fit(X_train,y_train)
# Predict
clf_predictions = clf.predict(X_test)
# Check For Accuracy

In [28]:
hamming_score(y_test, clf_predictions)

0.8057032009304846

In [29]:
def evaluate_model(y_test, br_prediction):
    score = f1_score(y_test,br_prediction,average="micro")
    precision = precision_score(y_test,br_prediction,average="micro")
    recall = recall_score(y_test,br_prediction,average="micro")
    report = classification_report(y_test,br_prediction,digits=4)
    EMR = accuracy_score(y_test,br_prediction)
    acc = hamming_score(y_test,br_prediction)
    
    #classifaction_report_csv(report,precision,recall,score,0)
    print ('\n clasification report:\n', report)
    print ('F1 score:', score)
    print ('Recall:', recall)
    print ('Precision:', precision)
    print('EMR:', EMR)
    print ('Acc:', acc)
    print('Hamming Loss', hamming_loss(y_test,br_prediction))
 

In [30]:
binary_rel_clf = build_model(MultinomialNB(),BinaryRelevance,X_train,y_train,X_test,y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 clasification report:
               precision    recall  f1-score   support

           0     0.7000    0.8319    0.7603      4407
           1     0.7882    0.9814    0.8743      4775
           2     0.9948    0.9472    0.9704      4636
           3     0.4903    0.9820    0.6540      1665
           4     0.8295    0.8617    0.8453      4781
           5     0.5701    0.9259    0.7057      2226

   micro avg     0.7472    0.9141    0.8223     22490
   macro avg     0.7288    0.9217    0.8017     22490
weighted avg     0.7786    0.9141    0.8326     22490
 samples avg     0.8135    0.9180    0.8443     22490

F1 score: 0.8223106613603184
Recall: 0.9141396176078257
Precision: 0.7472467560789445
EMR: 0.692940108025429
Acc: 0.8057032009304846
Hamming Loss 0.07078214871819384


In [31]:
clf_chain_model = build_model(MultinomialNB(),ClassifierChain,X_train,y_train,X_test,y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 clasification report:
               precision    recall  f1-score   support

           0     0.7000    0.8319    0.7603      4407
           1     0.8604    0.9652    0.9098      4775
           2     0.9943    0.9349    0.9636      4636
           3     0.8413    0.9550    0.8945      1665
           4     0.9130    0.8214    0.8648      4781
           5     0.6176    0.9155    0.7376      2226

   micro avg     0.8249    0.8966    0.8592     22490
   macro avg     0.8211    0.9040    0.8551     22490
weighted avg     0.8423    0.8966    0.8639     22490
 samples avg     0.8520    0.8998    0.8653     22490

F1 score: 0.8592491583926365
Recall: 0.8965762561138284
Precision: 0.8249059073801341
EMR: 0.7789780603221643
Acc: 0.8439478673740899
Hamming Loss 0.05262654748816978


In [32]:
clf_labelP_model = build_model(MultinomialNB(),LabelPowerset,X_train,y_train,X_test,y_test)


 clasification report:
               precision    recall  f1-score   support

           0     0.8166    0.8275    0.8220      4407
           1     0.9840    0.9420    0.9626      4775
           2     0.9979    0.9023    0.9477      4636
           3     0.7314    0.7982    0.7634      1665
           4     0.8975    0.8061    0.8494      4781
           5     0.6802    0.9030    0.7759      2226

   micro avg     0.8756    0.8680    0.8718     22490
   macro avg     0.8513    0.8632    0.8535     22490
weighted avg     0.8869    0.8680    0.8747     22490
 samples avg     0.8728    0.8691    0.8691     22490

F1 score: 0.8717650999218488
Recall: 0.8679857714539796
Precision: 0.8755774837407491
EMR: 0.8517279288752928
Acc: 0.8650956773895448
Hamming Loss 0.045751477781495464
